# Production

This notebook shows how to generate the final classification function and how to use it.

### Generation of the function

We start with a regular `Detector` that loads all the data we gathered.


In [1]:
from detector import *
from loader import load_detector
detector = load_detector()

loaded 5646 rows (0.04 s)
using seed: 656558797


Then we feed it a classifier to train. We then store it back into the detector.

In [2]:
from sklearn import ensemble
from sklearn import tree

untrained_clf = ensemble.AdaBoostClassifier (
    base_estimator = tree.DecisionTreeClassifier(max_depth=4),
    n_estimators = 46,
    learning_rate = .6
)

trained_clf = detector.classify(untrained_clf)[2]
detector.best_clf = trained_clf

Now, we could use `detector.predict(class_, length, width)` for our predictions. But sharing the `Detector` object is not a good solution: it would also gather the (now useless) dataframes, other methods, etc. So instead, we just dump the method.

In [3]:
import pickle
file = open('anomaly_classifier.clf', 'wb')
pickle.dump(detector.predict, file)
file.close()

### Usage

Now, we just have to load the function and apply it. The file is easily shared as it only weighs about 400 Kb.

In [4]:
def classify(class_, length, width):
    import pickle
    file = open('anomaly_classifier.clf', 'rb')
    fun = pickle.load(file)
    file.close()
    if fun(class_, length, width):
        return "malicious"
    else:
        return "non-malicous"

print("A moto 6 meters long and 1 meter wide:", classify("moto", 6000, 1000))
print("A car 5 meters long and 2 meter wide:", classify("car", 5000, 2000))

A moto 6 meters long and 1 meter wide: malicious
A car 5 meters long and 2 meter wide: non-malicous
